In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from geneticalgorithm import geneticalgorithm as ga

In [2]:
def formular(X, A, a1, a2, a3, Beta):
    S, T = X
    coe = (a1 + a2 * S + a3 * np.log(S)) * (1 - Beta * (T -25))
    Pm = coe * A * S
    return Pm

In [3]:
i=4
train = pd.read_csv('D:/dissertation/dataset/pre_dataset/train_{i}.csv'.format(i=i))
train.shape

(33856, 170)

In [4]:
train_ = train[
    (train['实发辐照度'] >=0) & 
    (train['实发辐照度'].notna() &
     train['实际功率']>0)
]
S = train_['实发辐照度']
T = train_['温度.1']
power = train_['实际功率']

In [ ]:
def objective(params):
    A, a1, a2, a3, Beta = params
    Pm_pred = formular((S, T), A, a1, a2, a3, Beta)
    return np.mean((Pm_pred - power) ** 2)

# train1,2,3,4
varbounds = np.array([
    [0, 0.5],   # A
    [0, 1],         # a1
    [0, 1e-2],      # a2
    [0, 0.05],       # a3
    [0.003, 0.005]      # Beta
])


model = ga(function=objective, dimension=5, variable_type='real', variable_boundaries=varbounds, 
           algorithm_parameters={'max_num_iteration': 100,  # 增加最大迭代次数
                                'population_size':100,    # 增大种群规模（默认可能为50）   
                                'mutation_probability':0.2,  # 变异概率（默认0.1，可尝试0.05~0.2）
                                'elit_ratio':0.01,        # 保留精英个体的比例
                                'crossover_probability':0.7,  # 交叉概率（通常0.7~0.9）
                                'parents_portion':0.3,    # 选择父代的比例
                                'crossover_type':'uniform',  # 交叉方式（还可选'one_point'/'two_point'）
                                'max_iteration_without_improv':None
                                })
model.run()
A, a1, a2, a3, Beta = model.output_dict['variable']

In [ ]:
# if i == 1:
#     A, a1, a2, a3, Beta = 1.01598603e-02, 6.83690555e-01, 7.08087957e-06, 9.60380372e-03, 4.11147028e-03
# elif i==2: 
#     A, a1, a2, a3, Beta = 6.61551883e-03, 9.95830719e-01, 5.17413010e-05, 3.28085781e-02, 4.51979587e-03
# elif i==3: 
#     A, a1, a2, a3, Beta = 2.04982725e-02, 9.93145756e-01, 6.48898141e-04, 3.53278180e-02, 3.39234014e-03
# elif i==4: 
#     A, a1, a2, a3, Beta = 5.81885884e-02, 5.61132154e-01, 2.75889664e-05, 1.26467258e-02, 3.62124689e-03

In [7]:
train['pred_power_effi'] = 0
train.loc[train['实发辐照度'] > 0, 'pred_power_effi'] = formular(
    (train['实发辐照度'], train['温度.1']), A, a1, a2, a3, Beta
)
# mse
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(train['pred_power_effi'], train['实际功率'])
mse

d:\anaconda3\envs\dissertation\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


14.126623821088032

In [8]:
# # train = train.drop('温度.1', axis=1)
# train.to_csv('D:/dissertation/dataset/final_dataset/train_{i}.csv'.format(i=i), index=False)
# train.shape

In [9]:
# test
test = pd.read_csv('../dataset/pre_dataset/test_{i}.csv'.format(i=i)) 
test['pred_power_effi'] = 0  # 初始化为0

In [10]:
# 对于'实发辐照度'大于0的行，使用formular函数计算实际功率
test.loc[test['实发辐照度'] > 0, 'pred_power_effi'] = formular(
    (test['实发辐照度'], test['温度.1']), A, a1, a2, a3, Beta
)
# test = test.drop('温度.1', axis=1)
test.shape

d:\anaconda3\envs\dissertation\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


(8538, 171)

In [11]:
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(test['实际功率'], test['pred_power_effi'])
mae = mean_absolute_error(test['实际功率'], test['pred_power_effi'])
rmse = rmse(test['实际功率'], test['pred_power_effi'])
print(f'MSE: {mse:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}')

MSE: 46.4671, MAE: 4.9349, RMSE: 6.8167


In [13]:
# test.to_csv('D:/dissertation/dataset/final_dataset/test_{i}.csv'.format(i=i), index=False)